In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-quantum-simulation-active-graph-networks/t0.json


In [2]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
import networkx as nx
import plotly.graph_objects as go

# Reconstruct the Titanic JSON based on earlier discussions
titanic_base_node = {
    "id": "Titanic_Base",
    "name": "Captain's Bridge",
    "type": "Root",
    "description": "The central node representing the Titanic's core structure, connecting all ship elements hierarchically.",
    "layers": [
        {
            "id": "Temporal_0",
            "name": "Ship Structure",
            "description": "Static hierarchical representation of the Titanic, organized by levels and compartments.",
            "nodes": {
                "Deck_A": {
                    "type": "Level",
                    "description": "Top passenger deck"
                },
                "Deck_B": {
                    "type": "Level",
                    "description": "Second passenger deck"
                },
                "Engine_Room": {
                    "type": "Level",
                    "description": "Engine and boiler compartments"
                },
                "Lifeboat_Stations": {
                    "type": "Safety",
                    "description": "Lifeboat deployment stations"
                }
            },
            "relationships": [
                {"source": "Titanic_Base", "target": "Deck_A", "relationship_type": "parent_of"},
                {"source": "Deck_A", "target": "Deck_B", "relationship_type": "parent_of"},
                {"source": "Deck_B", "target": "Engine_Room", "relationship_type": "parent_of"},
                {"source": "Deck_A", "target": "Lifeboat_Stations", "relationship_type": "connects_to"}
            ]
        }
    ]
}

# Create a directed graph
G = nx.DiGraph()

# Add the base node (Captain's Bridge)
G.add_node("Titanic_Base", name="Captain's Bridge", type="Root", description="The central node representing the Titanic's core.")

# Add nodes from the first layer
nodes = titanic_base_node["layers"][0]["nodes"]
for node_id, attrs in nodes.items():
    G.add_node(node_id, **attrs)

# Add edges (relationships) from the first layer
relationships = titanic_base_node["layers"][0]["relationships"]
for relationship in relationships:
    G.add_edge(relationship["source"], relationship["target"], relationship_type=relationship["relationship_type"])

# Assign positions for the nodes in a 3D space (arbitrary layout for now)
pos = {
    "Titanic_Base": (0, 0, 0),
    "Deck_A": (0, 1, 1),
    "Deck_B": (0, 2, 2),
    "Engine_Room": (0, 3, 3),
    "Lifeboat_Stations": (1, 1, 1)
}

# Extract node positions and edges for visualization
node_x = [pos[node][0] for node in G.nodes]
node_y = [pos[node][1] for node in G.nodes]
node_z = [pos[node][2] for node in G.nodes]

edge_x = []
edge_y = []
edge_z = []
for edge in G.edges:
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]

# Create a 3D Plotly graph
fig = go.Figure()

# Add edges to the graph
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='blue', width=2),
    hoverinfo='none'
))

# Add nodes to the graph
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    marker=dict(size=10, color='red'),
    text=[f"{node}: {G.nodes[node].get('description', '')}" for node in G.nodes],
    textposition="top center"
))

# Set the layout for better visualization
fig.update_layout(
    title="Titanic Active Graph Network - Base Visualization",
    scene=dict(
        xaxis_title="X Axis",
        yaxis_title="Y Axis",
        zaxis_title="Z Axis"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()
